In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path

import pandas as pd
import numpy as np

In [3]:
cd ..

C:\Projects\python\recommender


In [4]:
import torch as T
import torch.nn as nn
import torch.optim as optim

from datasets import MovelenDataset, TorchMovielen10k

## Init Dataloader instance

In [5]:
DEVICE = T.device('cpu')
BATCH = 32
SHUFFLE = False
WORKERS = 0
FILE_PATH = Path("./inputs/ml-100k/u.data")

In [6]:
databunch = TorchMovielen10k(FILE_PATH, user_min=4, item_min=4)
databunch.batch(BATCH)
databunch.device(DEVICE)
databunch.shuffle(SHUFFLE)
databunch.workers(WORKERS)

2019-09-04 17:14:58,502 - C:\Projects\python\recommender\utils.py - INFO - Read dataset in inputs\ml-100k\u.data
I0904 17:14:58.502703  1640 torch_movielen.py:41] Read dataset in inputs\ml-100k\u.data
2019-09-04 17:14:58,512 - C:\Projects\python\recommender\utils.py - INFO - Original user size: 943
I0904 17:14:58.512646  1640 torch_movielen.py:45] Original user size: 943
2019-09-04 17:14:58,514 - C:\Projects\python\recommender\utils.py - INFO - Original item size: 1682
I0904 17:14:58.514641  1640 torch_movielen.py:46] Original item size: 1682
2019-09-04 17:14:58,520 - C:\Projects\python\recommender\utils.py - INFO - Filter user size: 943
I0904 17:14:58.520633  1640 torch_movielen.py:52] Filter user size: 943
2019-09-04 17:14:58,523 - C:\Projects\python\recommender\utils.py - INFO - Filter item size: 1413
I0904 17:14:58.523616  1640 torch_movielen.py:53] Filter item size: 1413
c:\projects\python\recommender\.venv\lib\site-packages\pandas\core\generic.py:8682: SettingWithCopyWarning: 
A 

In [7]:
train_dl = databunch.get_dataloader(dataset_type='train')
train_it = iter(train_dl)

In [8]:
users, pos_batch, neg_batch = train_it.next()
print("positive batch sample: {}".format(pos_batch.shape))
print("negative batch sample: {}".format(neg_batch.shape))
print("users shape: {}".format(users.shape))

positive batch sample: torch.Size([32, 3769])
negative batch sample: torch.Size([32, 3769])
users shape: torch.Size([32])


## Init and test FM model

In [9]:
feat_dim = databunch.feat_dim
num_dim = 32
init_mean = 1

In [10]:
from models import TorchFM, FMLearner

In [11]:
model = TorchFM(feat_dim, num_dim, init_mean)
model

TorchFM()

In [12]:
user_index, pos_feats, neg_feats = train_it.next()

In [13]:
pos_preds, neg_preds = model(pos_batch, neg_batch)

In [14]:
print(pos_preds.size())
print(neg_preds.size())

torch.Size([32])
torch.Size([32])


In [15]:
pos_preds.device

device(type='cpu')

## Init and test Learner functions

In [16]:
op = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(op, step_size=1000, gamma=1.)

In [17]:
learner = FMLearner(model, op, scheduler, databunch)

### Test Criteirion funcions

In [18]:
pos_preds, neg_preds = model(pos_feats, neg_feats)

In [19]:
l2_reg = learner.compute_l2_term(linear_reg=1.0, factor_reg=1.0)
l2_reg

tensor([41362.0430], grad_fn=<AddBackward0>)

In [20]:
bprloss = learner.criterion(pos_preds, neg_preds, linear_reg=0.001, factor_reg=0.001)
bprloss

tensor([361.5419], grad_fn=<NegBackward>)

### Test Accuracy Functions

In [21]:
user_index, pos_feats, neg_feats = train_it.next()

In [22]:
pos_preds, neg_preds = model(pos_feats, neg_feats)

In [23]:
learner.update_hit_counts(user_index, pos_preds, neg_preds)

In [24]:
users = T.unique(user_index)
users

tensor([711])

In [25]:
learner.hit_per_user[users]

tensor([11.])

In [26]:
learner.user_counts[users]

tensor([32.])

In [27]:
learner.compute_auc()

tensor(0.0004)

### Test Loop Functions

In [28]:
user_index, pos_feats, neg_feats = train_it.next()

In [30]:
user_index.dtype

torch.int64

In [31]:
op.zero_grad()

In [32]:
pos_preds, neg_preds = model(pos_feats, neg_feats)

In [39]:
print(pos_preds.dtype)
print(neg_preds.dtype)

torch.float64
torch.float64


In [34]:
bprloss = learner.criterion(pos_preds, neg_preds)

In [37]:
bprloss

tensor([219.6089], grad_fn=<NegBackward>)

In [38]:
bprloss.dtype

torch.float32

In [42]:
l2_reg = learner.compute_l2_term(linear_reg=0.1, factor_reg=0.1)
l2_reg

tensor([4136.2043], dtype=torch.float64, grad_fn=<AddBackward0>)

In [43]:
l2_reg.dtype

torch.float64